## TD 3 Risk Model

#### Exercise 1 : Risk model

The Capital Asset Pricing Model (CAPM) assumes that asset returns can be shaped with $\theta_i$ the specific risk of each asset : $\mu_i - \mu_f = \beta_i (\mu_m-\mu_f)+\epsilon_i$ where $\epsilon_i \sim \mathcal{N}(0, \theta_i^2)$

a. Advantages of CAPM compared to Markowitz procedure for obtaining a efficiently diversified portfolio : 
- Simplicity as it reduces the dimensionality of the problem by focusing on the systematic risk (market beta) instead of considering the full covariance matrix as in the Markowitz procedure. 
- Easier to implement as CAPM uses a single factor (market protfolio) which is easier to estimate than the numerous inputs required for the Markowitz procedure.
- Conceptual clarity : it provides a clear relationship between risk and expected return through the security market line.  

And its disadvantages : 
- Strict assumptions : market are efficient, no taxes or transactions costs, investors hold mean-variance-efficient portfolios that may not hold in reality. 
- Focus on systematic risk : CAPM ignoes the idiosyncratic risk, which could be significant in practice for non-diversified portfolios. 
- Real-world performance : empirical studies sometimes show deviations from CAPM predictions (size, value, momentum effects). 


b. Impact of firm-specific risk on active investors' decisions : 
- Fir-specific risk (denoted by $\theta$) doesn't affect the expected return in the CAPM framework because it is assumed to be diversifiable in a well-diverisifed portfolio (example in the slides of an equally weighted portfolio). 
- However, for an active investor managing a less diversified portfolio, higher firm-specified risk reduces the portfolio's Sharpe ratio because the overall risk increases without a corresponding increase in expected return. It makes the active investors less willing to deviate from the market portfolio. 

c. On $\alpha$ : 
- Alpha is called a non-market return premium as it measures the excess return of an asset or portfolio beyond what is predicted by the CAPM (market return). It represents the value added by active management or other factors not captured by market beta. 
- High-alpha stocks are desirable as they will provide excess returns, making them attractive to active portfolio managers seeking to outperform the market. 
- Impact on the Sharpe ratio if the $\alpha$ increases : the Sharpe ratio improves if the alpha of individual securities in the portfolio increases as the expected return increases, as long as other parameters remain constant, such as the risk. 


d. For mean-variance optimization of 60 securities : 
- Expected returns : 60 estimates (one for each security)
- Variances : 60 estimates
- Covariance : 60*(60-1)/2 = 1770.
- Total estimates = 1890 estimates.

If CAPM assumptions hold : 
- One market return estimate 
- 60 betas (systematic risk, relative to the market) for the securities. 
- Total estimates = 61 estimates which significantly reduces the number of estimates needed. 

#### Exercise 2 : Tactical views

$w_{rob} = argmax(\overline{\mu}^Tw- \kappa \sqrt{w^T \Omega w}-\frac{\lambda}{2}w^T \Sigma w)$ with $w_{rob}$ the weights of actif portfolio, $\overline{\mu}$ the expected returns, $\kappa$ the uncertainty on return, $\lambda$ the risk aversion, $\Sigma$ variance-covariance matrix and $\Omega$ the uncertainty matrix.   

In [ ]:
import numpy as np
from scipy.optimize import minimize
sigma = np.array([0.143, 0.164, 0.083, 0.07, 0.001])
rho = np.array([[1, 0.821, 0.285, -0.06, 0],
               [0.821,1,0.344,0.017, 0],
               [0.285, 0.344, 1, 0.589, 0],
               [0.06,0.017,0.589,1, 0],
               [0,0,0,0,1]])
w_eq = np.array([0.25,0.25,0.25,0.25,0])
kappa = 0.2
lamb = 13.3
risk_budget = 0.03
n = len(sigma)
Sigma = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        Sigma[i,j] = sigma[i]*sigma[j]*rho[i,j]
Omega = np.diag(np.diag(Sigma))

In [15]:
score_views = np.array([-0.25, 0.75, 0, 0.5, 0])
# Indeed, there is a slowdown in the US economy with a score of 25%, a recovery in the Euro Zone with a score of 75%, 
# and a fall in the Euro zone 10y rate with a score of 50%, that is to say an increase in the price of the bond.
mu = score_views * sigma

In [16]:
def objective(w):
    return -(mu.T @ w - kappa * np.sqrt(w.T @ Omega @ w) - lamb/2 * w.T @ Sigma @ w)

def tracking_error_constraint(w):
    return risk_budget - np.sqrt((w-w_eq).T @ Sigma @ (w-w_eq))

constraints = [{'type':'ineq', 'fun': tracking_error_constraint}]

result = minimize(objective, w_eq, constraints=constraints, method = "SLSQP")
w_rob = result.x
print("Unconstrained Tactical Portfolio Weights: ", w_rob)
print("Tracking Error: ", np.sqrt((w_rob-w_eq).T @ Sigma @ (w_rob-w_eq)))

Unconstrained Tactical Portfolio Weights:  [-0.05883234  0.45876509  0.02976658  0.40192185  0.        ]
Tracking Error:  0.030000079713092143


In [19]:
expected_return = kappa*np.linalg.inv(Omega) @ mu # as here the benchmark mu is set to the null vector.
print("Vector of expected returns from tactical views:", expected_return)

Vector of expected returns from tactical views: [-0.34965035  0.91463415  0.          1.42857143  0.        ]


In [21]:
def constraint_sum_to_one(w):
    return np.sum(w) - 1

def constraint_non_negative(w):
    return w

# Constraints definition for scipy.optimize.minimize
constraints = ({'type':'ineq', 'fun': tracking_error_constraint},
               {'type': 'eq', 'fun': constraint_sum_to_one},
                {'type': 'ineq', 'fun': constraint_non_negative})

result_final = minimize(objective, w_eq, constraints=constraints, method = "SLSQP")
w_rob_final = result_final.x
print("Unconstrained Tactical Portfolio Weights: ", w_rob_final)
print("Tracking Error compared to the benchmark (equal to 0 by definition): ", np.sqrt((w_rob_final-w_eq).T @ Sigma @ (w_rob_final-w_eq)))

Unconstrained Tactical Portfolio Weights:  [-7.88617887e-17  3.77907039e-01  1.40351425e-16  4.23815964e-01
  1.98276997e-01]
Tracking Error compared to the benchmark (equal to 0 by definition):  0.030000001436052352
